In [6]:
# !conda install gensim
#NOTE: See this medium article that I drew on heavily for this solution:
#https://medium.com/betacom/unsupervised-nlp-task-in-python-with-doc2vec-da1f7727857d

import pandas as pd
import gensim
from gensim.parsing.preprocessing import preprocess_documents
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [50]:
#reduce size of datasets to push to github
# scraped_urls = pd.read_csv('DATASETS/podium_location_urls copy.csv')
# scraped_urls = scraped_urls.sample(100)
# urls_for_scrap = pd.read_csv('DATASETS/urls_for_scraping copy.csv')
# urls_for_scrap = urls_for_scrap.sample(100)

# scraped_urls.to_csv('DATASETS/podium_location_urls copy.csv')
# urls_for_scrap.to_csv('DATASETS/urls_for_scraping copy.csv')

In [7]:
import pandas as pd
location_urls = pd.read_csv('./DATASETS/podium_location_urls copy.csv',delimiter=',', encoding="utf-8", skipinitialspace=True)
location_urls.drop_duplicates(subset=['url'],inplace=True)
# still not right merge
print(len(location_urls))
print(location_urls.columns)

22902
Index(['Unnamed: 0', 'Account_Name', 'Org_UID', 'Opportunity_Name', 'Type',
       'url', 'raw_html_text'],
      dtype='object')


In [8]:
len(location_urls)

22902

In [9]:
print(location_urls.head())
len(location_urls)

   Unnamed: 0             Account_Name Org_UID Opportunity_Name Type  \
0           0              Jody Savage     NaN              NaN  NaN   
1           1  Mountain View Financial     NaN              NaN  NaN   
2           2                    fhgjh     NaN              NaN  NaN   
3           3         Focus Floors LLC     NaN              NaN  NaN   
4           4         Maplewood Senior     NaN              NaN  NaN   

                                    url  \
0            http://www.jodysavage.com/   
1  http://www.mountainviewfinancial.com   
2              http://www.naluzotan.com   
3              https://focusfloors.net/   
4  http://www.maplewoodseniorliving.com   

                                       raw_html_text  
0  \n \n \n \n \n \n \n \n \n \n \n \n \n \n Sava...  
1  \n \n \n \n \n \n \n \n \n \n \n \n connectFir...  
2                                        no response  
3  Focus Floors \n \n \n \n (614) 751-2117 (614) ...  
4  \n 403 Forbidden \n \n 403 For

22902

In [10]:
location_urls = location_urls.loc[location_urls['raw_html_text'].notna()]
location_urls = location_urls.loc[~location_urls['raw_html_text'].isin(['no response'])]
location_urls = location_urls.loc[~location_urls['raw_html_text'].str.contains('Access was denied')]

In [11]:
location_urls['raw_html_text'] = location_urls['raw_html_text'].str.replace("\n","")
location_urls['raw_html_text'] = location_urls['raw_html_text'].str.replace("\t","")
location_urls['raw_html_text'] = location_urls['raw_html_text'].str.replace("\r","")
location_urls['raw_html_text'] = location_urls['raw_html_text'].str.replace("\ufeff","")
location_urls['raw_html_text'] = location_urls['raw_html_text'].str.split()
location_urls['raw_html_text'] = [" ".join(x) for x in location_urls['raw_html_text']]

In [12]:
len(location_urls.loc[location_urls['raw_html_text'].str.len()<50])
location_urls.loc[location_urls['raw_html_text'].str.len()<50,'raw_html_text'].head()
location_urls = location_urls.loc[location_urls['raw_html_text'].str.len()>50].reset_index(drop=True)

In [13]:
len(location_urls)

15450

In [14]:
location_urls['raw_html_text'].loc[0]

'Savage Real Estate Group - Savage Real Estate Group Skip to content Savage Real Estate Group Serving Bozeman and the Gallatin Valley Home Featured Listings Listings 3D Virtual Home Tours 406.209.8884 Buyers Sellers Market Stats About Us Testimonials Bozeman Info Articles Contact Us Savage Real Estate Group Experienced Real Estate Brokerage in Bozeman, MT Serving Bozeman, Belgrade, Livingston and the Gallatin Valley View listings by city: Bozeman | Belgrade | Livingston | Three Forks $1,499,000 49 Riparian Way Bozeman, Montana 5 Beds | 5 Baths | 0.946 Acres $1,450,000 1984 Vaquero Parkway Bozeman, Montana 4 Beds | 3 Baths | 0.1583 Acres $1,425,000 433 Princeton Place Bozeman, Montana 5 Beds | 4 Baths | 0.2738 Acres $1,400,000 107 Kimber Court Bozeman, Montana 1 Bed | 1 Bath | 7.44 Acres $1,399,000 513 Cambridge Drive Bozeman, Montana 6 Beds | 4 Baths | 0.2915 Acres $1,375,000 5190 Copeland Bozeman, Montana 4 Beds | 3 Baths | 1.206 Acres $1,365,000 4616 Danube Lane Bozeman, Montana 4 Be

In [15]:
location_urls.columns
location_urls.drop(columns=['Unnamed: 0'], inplace=True)
location_urls.index.name = 'index'

In [16]:
location_urls['raw_html_text'].head(1)

index
0    Savage Real Estate Group - Savage Real Estate ...
Name: raw_html_text, dtype: object

In [17]:
text_corpus = location_urls['raw_html_text'].values

In [18]:
text_corpus[0:1]

array(['Savage Real Estate Group - Savage Real Estate Group Skip to content Savage Real Estate Group Serving Bozeman and the Gallatin Valley Home Featured Listings Listings 3D Virtual Home Tours 406.209.8884 Buyers Sellers Market Stats About Us Testimonials Bozeman Info Articles Contact Us Savage Real Estate Group Experienced Real Estate Brokerage in Bozeman, MT Serving Bozeman, Belgrade, Livingston and the Gallatin Valley View listings by city: Bozeman | Belgrade | Livingston | Three Forks $1,499,000 49 Riparian Way Bozeman, Montana 5 Beds | 5 Baths | 0.946 Acres $1,450,000 1984 Vaquero Parkway Bozeman, Montana 4 Beds | 3 Baths | 0.1583 Acres $1,425,000 433 Princeton Place Bozeman, Montana 5 Beds | 4 Baths | 0.2738 Acres $1,400,000 107 Kimber Court Bozeman, Montana 1 Bed | 1 Bath | 7.44 Acres $1,399,000 513 Cambridge Drive Bozeman, Montana 6 Beds | 4 Baths | 0.2915 Acres $1,375,000 5190 Copeland Bozeman, Montana 4 Beds | 3 Baths | 1.206 Acres $1,365,000 4616 Danube Lane Bozeman, Monta

In [19]:
processed_corpus = preprocess_documents(text_corpus)
tagged_corpus = [TaggedDocument(d, [i]) for i, d in enumerate(processed_corpus)]

In [20]:
tagged_corpus[0:1]

[TaggedDocument(words=['savag', 'real', 'estat', 'group', 'savag', 'real', 'estat', 'group', 'skip', 'content', 'savag', 'real', 'estat', 'group', 'serv', 'bozeman', 'gallatin', 'vallei', 'home', 'featur', 'list', 'list', 'virtual', 'home', 'tour', 'buyer', 'seller', 'market', 'stat', 'testimoni', 'bozeman', 'info', 'articl', 'contact', 'savag', 'real', 'estat', 'group', 'experienc', 'real', 'estat', 'brokerag', 'bozeman', 'serv', 'bozeman', 'belgrad', 'livingston', 'gallatin', 'vallei', 'view', 'list', 'citi', 'bozeman', 'belgrad', 'livingston', 'fork', 'riparian', 'wai', 'bozeman', 'montana', 'bed', 'bath', 'acr', 'vaquero', 'parkwai', 'bozeman', 'montana', 'bed', 'bath', 'acr', 'princeton', 'place', 'bozeman', 'montana', 'bed', 'bath', 'acr', 'kimber', 'court', 'bozeman', 'montana', 'bed', 'bath', 'acr', 'cambridg', 'drive', 'bozeman', 'montana', 'bed', 'bath', 'acr', 'copeland', 'bozeman', 'montana', 'bed', 'bath', 'acr', 'danub', 'lane', 'bozeman', 'montana', 'bed', 'bath', 'acr',

In [21]:
#NOTE: Full transparency I bought a linear algebra course to try and understand all of the math behind this 
#and to date I only understand it at a high level but essentially what makes Doc2Vec (aka Paragraph Vector) so special
#is that it uses two different architectures respectively called Distributed Memory and Distributed Bag of Words
#which essentially build on similar concepts found in the Word2Vec algorithm and form what are known as context windows
#to be able to remember what words frequently appear next to each other and essentially allow the model to understand
#the semantics of a sentence which is something computers traditionally can't do.
#(e.g. to know that "powerful" and "strong" and "Paris" are not equally distant. See the abstract put forth by
# Doc2Vec creators found here: https://arxiv.org/pdf/1405.4053v2.pdf)

import pickle
model = Doc2Vec(tagged_corpus, dm=0, vector_size=200, window=2, min_count=1, epochs=100, hs=1)
pickle.dump(model, open("./MODELS/podium_url_doc2vec_v1 copy.dat", "wb"))

# model = pickle.load(open('./MODELS/podium_url_doc2vec_v1 copy.dat', "rb"))

In [22]:
#NOTE: Because this was an unsupervised model there wasn't a train/test set to check accuracy so I scraped 10 random websites
#that fit into one of our subverts without knowledge of if they were in the original list of urls or not and 
#ran them against the model which returned the most similar documents. I then got the counts of most common words found
#between each document to be able to gut-check if the results seemed to be accurate. For sake of time I will skip the 
#code explanation

In [23]:
#Get some new urls for testing:
import requests
from bs4 import BeautifulSoup
def scraper (url):
    try:
        page = requests.get(url,timeout=5)
        x = BeautifulSoup(page.text, 'html.parser').get_text(separator=' ')
        print(f"{url} scraped successfully")
    except Exception as e:
        print(e)
        raise e
        x = 'no response'
    return x

In [24]:
urls = ['https://www.automotiverepairportland.com/auto-body-collision-repair.php','https://classiccollision.net/','https://www.clawsonheating.com/',
'https://www.bickmoreauto.com/','https://www.greshamford.com/','https://www.greshamroofing.com/','https://allphaseexterior.com/',
'https://www.ottossausage.com/','https://pawntalk.com/','https://www.furnitureconnexion.com/']
output = []
for u in urls:
    l = scraper(u)
    output.append(l)
scraped = pd.Series(output)
scraped.rename('raw_html_text',inplace=True)

https://www.automotiverepairportland.com/auto-body-collision-repair.php scraped successfully
https://classiccollision.net/ scraped successfully
https://www.clawsonheating.com/ scraped successfully
https://www.bickmoreauto.com/ scraped successfully
https://www.greshamford.com/ scraped successfully
https://www.greshamroofing.com/ scraped successfully
https://allphaseexterior.com/ scraped successfully
https://www.ottossausage.com/ scraped successfully
https://pawntalk.com/ scraped successfully
https://www.furnitureconnexion.com/ scraped successfully


0    \n \n \n \n Auto Body & Collision Repair in Po...
1    \n \n \n \n \n \n \n \n Auto Body and Collisio...
2    \n \n \n Portland, OR Heating & Air Conditioni...
3    \n \n \n \n \n \n Bickmore Auto Sales - New & ...
4              Gresham Ford | Oregon Ford Dealer | ...
5    \n \n \n \n \n \n \n \n \n \n \n \n Gresham Ro...
6    \n 403 Forbidden \n \n 403 Forbidden \n nginx ...
7    Warning: Gravity Forms is not installed or act...
8    \n \n \n \n \n \n \n \n \n Silver Lining Jewel...
9    \n \n \n \n \n \n \n \n \n \n \n Furniture Con...
Name: raw_html_text, dtype: object

In [25]:
scraped = scraped.loc[scraped.notna()]
scraped = scraped.loc[~scraped.isin(['no response'])]
scraped = scraped.loc[~scraped.str.contains('Access was denied')]

In [26]:
scraped = scraped.str.replace("\n","")
scraped = scraped.str.replace("\t","")
scraped = scraped.str.replace("\r","")
scraped = scraped.str.split()
scraped = [" ".join(x) for x in scraped]
scraped = pd.Series(scraped)

In [27]:
scraped = scraped.loc[scraped.str.len()>50].reset_index(drop=True)
# scraped.loc[1]

In [28]:
index_lst = []
def get_best_score(df, s):
    new_doc = s
    new_doc = gensim.parsing.preprocessing.preprocess_string(new_doc)
    test_doc_vector = model.infer_vector(new_doc)
    sims = model.dv.most_similar(positive = [test_doc_vector])
    index = sims[0][0]
    index_similarity_score = sims[0][1]
    df.loc[index,'index_similarity_score'] = index_similarity_score
    df.loc[index,'new_scraped_test_data'] = str(s)
    index_lst.append(index)
    return df

for z in scraped:
    get_best_score(location_urls, z)

sample_df = location_urls.loc[index_lst]
sample_df.columns
sample_df['new_scraped_test_data']


# for s in sims:
    # print(f"{(s[1])} | {location_urls['raw_html_text'].iloc[s[0]]}")

index
14999    Auto Body & Collision Repair in Portland, OR a...
1722     Auto Body and Collision Repair - Classic Colli...
10130    Portland, OR Heating & Air Conditioning | HVAC...
9008     Bickmore Auto Sales - New & Used Cars, Service...
15031    Gresham Ford | Oregon Ford Dealer | Serving Gr...
8744     Gresham Roofing and Construction | Gresham, OR...
10267    Warning: Gravity Forms is not installed or act...
8978     Silver Lining Jewelry and Loan - Oregon's Frie...
8847     Furniture Connexion | Portland Furniture Store...
Name: new_scraped_test_data, dtype: object

In [30]:
sample_df.index.name = 'index'

In [31]:
indices = sample_df.index.values
sample_df

,Account_Name,Org_UID,Opportunity_Name,Type,url,raw_html_text,index_similarity_score,new_scraped_test_data
index,,,,,,,,
14999,"Pronto Body Shop, Inc.",NaN,NaN,NaN,https://prontobodyshop.com/,Pronto Body Shop: Auto Collision Repair & Pain...,0.522350,"Auto Body & Collision Repair in Portland, OR a..."
1722,"Crystal Lake Automotive, Inc.",NaN,Crystal Lake Automotive,New Business,http://www.crystallakeautomotive.com,Auto Body and Collision Repair - Classic Colli...,0.945264,Auto Body and Collision Repair - Classic Colli...
10130,Holder - Inbound Account G,NaN,NaN,NaN,http://www.nicholsonsheatingcooling.com,Pine Bluff HVAC Company | Professional Heating...,0.609336,"Portland, OR Heating & Air Conditioning | HVAC..."
9008,Big Sky Motorsports,NaN,NaN,NaN,https://www.bigskymotorsports.com/,Big Sky Motorsports | Powersports Dealership i...,0.407272,"Bickmore Auto Sales - New & Used Cars, Service..."
15031,AMSI Automotive Group,NaN,NaN,NaN,http://www.athensford.com,Athens Ford | New & Used Ford Dealer in Athens...,0.607731,Gresham Ford | Oregon Ford Dealer | Serving Gr...
8744,R & R Roofing Inc,NaN,NaN,NaN,https://www.rnrroofing.com/,Roofing Company Wilmington DE | R & R Roofing ...,0.457915,"Gresham Roofing and Construction | Gresham, OR..."
10267,The Fresh Market,NaN,NaN,NaN,http://www.thefreshmarket.com,The Fresh Market | Delicious Easy Meals | Seas...,0.347840,Warning: Gravity Forms is not installed or act...
8978,The Vault Jewelry and Loan,NaN,The Vault Jewelry,New Business,https://thevaultloans.com/?utm_source=google&u...,Luxury Pawn Shop in VA | The Vault Jewelry & L...,0.493470,Silver Lining Jewelry and Loan - Oregon's Frie...
8847,Virginia Furniture Market,NaN,NaN,NaN,http://www.vafurnituremarket.com,"Virginia Furniture Market | Rocky Mount, Roano...",0.496830,Furniture Connexion | Portland Furniture Store...


In [32]:
# from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from gensim.parsing.preprocessing import remove_stopwords
import numpy as np

def get_tfidf_top_10(df,i):
    s = df['raw_html_text'].loc[i].split()
    x = df['new_scraped_test_data'].loc[i].split()
    lst = []
    for y in s: 
            if y in x:
                lst.append(y)
    return lst

mast_lst = []
for i in indices:
    lst = get_tfidf_top_10(sample_df,i)
    lst = [remove_stopwords(x) for x in lst]
    lst = Counter(lst).most_common(10)
    lst = pd.Series(lst)
    mast_lst.append(lst)
sample_df['most_common_words_between_docs'] = mast_lst



In [33]:
sample_df.to_csv('./DATASETS/embeddings_and_most_common_words.csv')

In [34]:
#NOTE: Potential use cases of this that I see are:
#1. Help with identifying patterns of top converting businesses within desireable strikezones to better target
#our lead scoring efforts
#2. Automating sub-vertical assignment/fixing incorrectly assigned sub-verts on our contacts
#3. Help with deduping to find the exact website or similar websites between new records coming in and existing records

#Thanks for listening!
